### imports

In [4]:
import pyvista as pv
import pymeshfix
import os
import trimesh
import numpy as np

##### Augmentation: mirror the mesh

In [5]:
def mirror_mesh(input_path):
    mesh = pv.read(input_path)
    # mirror the mesh on the Y axis
    mesh.points[:, 1] = -mesh.points[:, 1]
    mesh.save(input_path.replace(".ply", "_mirrored.ply"))
#
# meshPathes = [mesh for mesh in os.listdir("../experimentation_scripts/data_samples/train/") if mesh.endswith(".ply")]
#
# for mesh in meshPathes:
#     mesh_path = os.path.join("../experimentation_scripts/data_samples/train/", mesh)
#     mirror_mesh(mesh_path)

##### Fill holes in the mesh and clean the isolated vertices function

In [6]:
def fill_mesh_holes_pyvista(input_path, output_path, hole_size=1000.0):
    pymeshfix.clean_from_file(input_path, output_path)
    mesh = pv.read(output_path)
    mesh = mesh.clean(tolerance=1e-6)
    mesh = mesh.triangulate()
    # Fill holes
    filled = mesh.fill_holes(hole_size)

    # Optional: clean up
    filled = filled.clean()

    # Save the result
    filled.save(output_path)

    mesh = pv.read(output_path)
    # mirror the mesh on the Y axis
    # mesh.points[:, 1] = -mesh.points[:, 1]
    # mesh.save(output_path.replace(".ply", "_mirrored.ply"))

    return filled

##### preprocess the mesh and slice the back

In [7]:
data_path = "../experimentation_scripts/data_samples/train/2021 - 21 Pelinsu Özkan (Aktif).ply"
sliced_mesh_path = "./sliced_meshes"

# mesh_paths = [path for path in os.listdir(data_path) if path.endswith(".ply")]

mirrored_guys = True
def processMesh(meshPath: str):
    # fill the mesh using pyvista
    output_dir = os.path.join(os.path.dirname(meshPath), "filled_meshes")
    output_path = os.path.join(output_dir, os.path.splitext(os.path.basename(meshPath))[0] + ".ply")

    fill_mesh_holes_pyvista(meshPath, output_path, hole_size=10000.0)

    mesh = trimesh.load(output_path)
    center_z = (mesh.bounds[0][2] + mesh.bounds[1][2]) / 2
    # cut the mesh from with a plane on center_z
    plane_normal = [0, 0, 1]
    plane_origin = [0, 0, center_z]

    # Slice the mesh with the plane
    sliced_meshes = mesh.slice_plane(plane_origin, plane_normal)
    print(type(sliced_meshes))
    # sliced_meshes.export(f"{sliced_mesh_path}/{"".join(base_name.split(".")[:-1])}.ply")
    return sliced_meshes
mesh = processMesh(data_path)


<class 'trimesh.base.Trimesh'>


##### horizontally slice function

In [8]:
N_SLICES = 21

def sliceMesh(mesh):
    # cut the mesh across the Z axis alonge the Y axis
    jump = abs(mesh.bounds[0][0] - mesh.bounds[1][0]) / N_SLICES
    initial = mesh.bounds[0][0]
    plane_normal = [1, 0, 0]
    plane_origin = [initial, 0, 0]
    plane_negative_normal = [-1, 0, 0]

    # new_center_x = 0
    # Slice the mesh with the plane
    slices = []
    for i in range (N_SLICES):
        sliced_meshes = mesh.slice_plane(plane_origin, plane_normal)
        plane_origin[0] = plane_origin[0] + jump
        sliced_meshes = sliced_meshes.slice_plane(plane_origin, plane_negative_normal)
        slices.append(sliced_meshes)
        # sliced_meshes.export(f"{i}.ply")
    # remove the last N_SLICES - 17
    for i in range(N_SLICES - 17):
        slices.pop()
    return slices

# slices = sliceMesh(sliced_mesh)

##### point rotation function

In [9]:
from scipy.spatial.transform import Rotation as R
def rotate_normal_around_axis(normal, origin, axis, angle_degrees):
    axis = axis / np.linalg.norm(axis)  # ensure unit axis
    angle_rad = np.deg2rad(angle_degrees)
    rot = R.from_rotvec(angle_rad * axis)
    new_normal = rot.apply(normal)
    return new_normal

##### radial slice function

In [10]:
N_PATCHES = 10
def radial_slice(mesh : trimesh.Trimesh):
    # cut the mesh radially
    # print(len(mesh.vertices))
    initial_z = mesh.bounds[0][2]
    # print(f"Center X: {initial_z}")
    plane_normal = [0, 0, 1]
    plane_negative_normal = [0, 0, -1]
    # TODO TRY to take abs for (mesh.bounds[0][1] + mesh.bounds[1][1])
    plane_origin = np.array([0, (mesh.bounds[0][1] + mesh.bounds[1][1])/ 2, initial_z])
    axis = np.array([1, 0, 0])      # rotate around X
    # last_normal = rotate_normal_around_axis(np.array(plane_normal), np.array(plane_origin), axis, 170)
    # last_slice =  mesh.slice_plane(plane_origin, last_normal)
    # last_slice.export("last_slice.ply")


    patches_list = []

    for i in range(N_PATCHES):
        sliced_patches = mesh.slice_plane(plane_origin, plane_normal)
        plane_normal = rotate_normal_around_axis(np.array(plane_normal), np.array(plane_origin), axis, 180 / N_PATCHES)
        plane_negative_normal = rotate_normal_around_axis(np.array(plane_negative_normal), np.array(plane_origin), axis, 180 / N_PATCHES)
        sliced_patches = sliced_patches.slice_plane(plane_origin, plane_negative_normal)
        # sliced_patches.export(f"radial_slice_{i}.ply")
        patches_list.append(sliced_patches)

    return patches_list

# patches = radial_slice(slices[2])

In [11]:
counter = 0

##### get the normal of the patch through averaging the normals of the vertices

In [12]:
def get_patch_normal(patch : trimesh.Trimesh):
    if patch is None:
        print("Warning Patch is None")
    else:
        normals = np.mean(patch.vertex_normals, axis = 0)
        norm = np.linalg.norm(normals)
        if norm > 1e-6:
            normals = normals / norm
    if patch.vertex_normals.shape == (0,3):
        print(patch)
    return normals
# get_patch_normal(patches[0])

In [13]:
from skspatial.objects import Plane
def get_patch_normal_scikit_spatial(patch: trimesh.Trimesh):
    """
    Compute the normal of a mesh patch using scikit-spatial.

    Parameters:
    -----------
    patch : trimesh.Trimesh
        A mesh patch for which to estimate the normal

    Returns:
    --------
    numpy.ndarray
        The estimated normal vector for the patch
    """
    if patch is None:
        print("Warning: Patch is None")
        return np.array([0.0, 0.0, 1.0])  # Default normal

    # Check if the patch has enough vertices (need at least 3 points for a proper plane)
    if len(patch.vertices) < 3:
        print(f"Warning: Patch has only {len(patch.vertices)} vertices, need at least 3 for reliable PCA")
        if len(patch.vertices) == 0:
            return np.array([0.0, 0.0, 1.0])  # Default normal if no vertices
        elif len(patch.vertices) == 1 or len(patch.vertices) == 2:
            # Single point - use vertex normal if available, otherwise default
            avg_normal = np.mean(patch.vertex_normals, axis=0)
            return avg_normal / np.linalg.norm(avg_normal)

    # Get vertices as points for PCA
    points = patch.vertices
    plane = Plane.best_fit(points)

    # Get the normal of the plane
    normal = plane.normal

    # Ensure the normal has unit length
    normal = normal / np.linalg.norm(normal)
    # Optional: Make sure the normal points "outward"
    # We can use the average vertex normal as a reference for consistent orientation
    if len(patch.vertex_normals) > 0:
        avg_vertex_normal = np.mean(patch.vertex_normals, axis=0)
        avg_vertex_normal = avg_vertex_normal / np.linalg.norm(avg_vertex_normal)

        # If the computed normal points in the opposite direction of the average vertex normal,
        # flip it to maintain consistent orientation
        if np.dot(normal, avg_vertex_normal) < 0:
            normal = -normal

    return normal


In [14]:
import numpy as np
import trimesh
from scipy import linalg

def get_patch_normal_pca(patch: trimesh.Trimesh):
    """
    Compute the normal of a mesh patch using Principal Component Analysis (PCA).

    Parameters:
    -----------
    patch : trimesh.Trimesh
        A mesh patch for which to estimate the normal

    Returns:
    --------
    numpy.ndarray
        The estimated normal vector for the patch
    """
    if patch is None:
        print("Warning: Patch is None")
        return np.array([0.0, 0.0, 1.0])  # Default normal

    # Check if the patch has enough vertices (need at least 3 points for a proper plane)
    if len(patch.vertices) < 3:
        print(f"Warning: Patch has only {len(patch.vertices)} vertices, need at least 3 for reliable PCA")
        if len(patch.vertices) == 0:
            return np.array([0.0, 0.0, 1.0])  # Default normal if no vertices
        elif len(patch.vertices) == 1 or len(patch.vertices) == 2:
            # Single point - use vertex normal if available, otherwise default
            avg_normal = np.mean(patch.vertex_normals, axis=0)
            return avg_normal / np.linalg.norm(avg_normal)

    # Get vertices as points for PCA
    points = patch.vertices

    # Center the points by subtracting the mean
    points_centered = points - np.mean(points, axis=0)

    # Compute the covariance matrix
    cov_matrix = np.cov(points_centered, rowvar=False)

    # Check for numerical stability
    if np.any(np.isnan(cov_matrix)):
        print("Warning: NaN values in covariance matrix")
        if len(patch.vertex_normals) > 0:
            avg_normal = np.mean(patch.vertex_normals, axis=0)
            return avg_normal / np.linalg.norm(avg_normal)
        return np.array([0.0, 0.0, 1.0])

    # Compute eigenvalues and eigenvectors
    try:
        eigenvalues, eigenvectors = linalg.eigh(cov_matrix)
    except np.linalg.LinAlgError:
        print("Warning: Eigenvalue computation failed")
        if len(patch.vertex_normals) > 0:
            avg_normal = np.mean(patch.vertex_normals, axis=0)
            return avg_normal / np.linalg.norm(avg_normal)
        return np.array([0.0, 0.0, 1.0])

    # The normal is the eigenvector corresponding to the smallest eigenvalue
    # (the direction with least variance)
    normal = eigenvectors[:, 0]

    # Ensure the normal has unit length
    normal = normal / np.linalg.norm(normal)

    # Optional: Make sure the normal points "outward"
    # We can use the average vertex normal as a reference for consistent orientation
    if len(patch.vertex_normals) > 0:
        avg_vertex_normal = np.mean(patch.vertex_normals, axis=0)
        avg_vertex_normal = avg_vertex_normal / np.linalg.norm(avg_vertex_normal)

        # If the computed normal points in the opposite direction of the average vertex normal,
        # flip it to maintain consistent orientation
        if np.dot(normal, avg_vertex_normal) < 0:
            normal = -normal

    return normal


# Example usage:
# normal = get_patch_normal_pca(patches[0])

##### Get the features of the mesh

In [15]:
def get_features(meshPath, export_output=False):
    print(meshPath)
    back_slice = processMesh(meshPath)
    vertical_slices = sliceMesh(back_slice)

    # # collection of geometries
    geometries_original = []
    geometries_features = []

    all_normals = []
    for slice in vertical_slices:
        if slice is None:
            print("Warning vertical slice is None")
            continue
        patches = radial_slice(slice)
        for patch in patches:
            # make the original geometry
            if export_output:
                geometry_original = trimesh.Trimesh(vertices=patch.vertices, faces=patch.faces, process=False)
                geometry_original.vertex_normals = patch.vertex_normals
                geometries_original.append(geometry_original)

            patch_normal = get_patch_normal_scikit_spatial(patch)
            if export_output:
                # make the features geometry
                geometry_features = trimesh.Trimesh(vertices=patch.vertices, faces=patch.faces, process=False)
                like_array = np.full_like(geometry_features.vertex_normals, patch_normal)
                geometry_features.vertex_normals = like_array
                # geometry_features.vertex_normals = np.tile(patch_normal, (geometry_features.vertices.shape[0], 1))
                geometries_features.append(geometry_features)

            all_normals.append(patch_normal)

    if export_output:
        # export the original geometries as one unit
        merged = trimesh.util.concatenate(geometries_original)
        merged.export(meshPath.replace(".ply", "_original.ply"))
        # export the features geometries as one unit
        merged_features = trimesh.util.concatenate(geometries_features)
        merged_features.export(meshPath.replace(".ply", "_features.ply"))

    return np.array(all_normals)

In [16]:
# path = "../experimentation_scripts/data_samples/train/2021 - 21 Pelinsu Özkan (Aktif).ply"
# normals = get_features(path, export_output=False)

In [20]:
# path = "../experimentation_scripts/data_samples/train/kerem celik.ply"
# normals = get_features(path)

In [17]:
meshPathes = [mesh for mesh in os.listdir("../experimentation_scripts/data_samples/corrupted") if mesh.endswith(".ply")]

#### Extract features from the dataset

In [23]:
import pickle

features = []
for mesh in meshPathes:
    feature = get_features(os.path.join("../experimentation_scripts/data_samples/corrupted", mesh))
    feature = np.nan_to_num(feature)
    features.append((mesh, feature))
    print(len(features))

with open("features.pkl", "wb") as f:
    pickle.dump(features, f)

../experimentation_scripts/data_samples/corrupted/ARTI_ali_riza_ersin_(+).ply
1
../experimentation_scripts/data_samples/corrupted/2024-09-16_14-55-22_回复__visit_to_Germany.ply
2
../experimentation_scripts/data_samples/corrupted/ARTI_Sebnem_Ela_Cengizhan_(+).ply
3
../experimentation_scripts/data_samples/corrupted/AKTIF__azra_duru_yildiz.ply
4
../experimentation_scripts/data_samples/corrupted/2024-07-05_18-59-35_Fwd__Fw__Patient_Data_ziad_Kareem_3_case_5724.ply
5
../experimentation_scripts/data_samples/corrupted/2024-09-20_16-42-31_Fwd__Fw__Patient_Data_Maria_charity_1924.ply
6
../experimentation_scripts/data_samples/corrupted/ilkim cemal kara 140921KALECAN.ply
7
../experimentation_scripts/data_samples/corrupted/ARTI_yigit_ali_boz_2_(+).ply
8
../experimentation_scripts/data_samples/corrupted/KALECAN_ecrin_ada_sener-_korse.ply
9
../experimentation_scripts/data_samples/corrupted/ARTI_ayse_beren_nebil.ply
10
../experimentation_scripts/data_samples/corrupted/2023-10-06_10-11-52_Fw__3D_Model_J

In [17]:
# features[17] = (features[17][0], np.nan_to_num(features[17][1]))

In [18]:
features[293][1]

array([[ 9.21690895e-01, -1.89708578e-01,  3.38373388e-01],
       [ 9.16548315e-01, -2.64743872e-01,  2.99749677e-01],
       [ 9.97222801e-01, -2.92462576e-02, -6.84933639e-02],
       [ 6.16545871e-01,  6.78545813e-01, -3.99307863e-01],
       [ 9.44386483e-01,  3.19225679e-01, -7.89248794e-02],
       [ 9.94707859e-01, -9.36058920e-02, -4.23581500e-02],
       [ 9.62330852e-01, -2.22343823e-01, -1.56469026e-01],
       [ 9.99652565e-01, -1.78987380e-02, -1.93490064e-02],
       [ 7.49070047e-01, -2.97743046e-01, -5.91813436e-01],
       [ 8.96009880e-01, -6.15451295e-02,  4.39748215e-01],
       [ 6.56902768e-01, -6.97743590e-01,  2.85714256e-01],
       [ 5.46122903e-01, -7.72252629e-01, -3.24616161e-01],
       [ 1.39718471e-01, -2.04561965e-01, -9.68830817e-01],
       [ 2.82103944e-01, -2.26845478e-02, -9.59115622e-01],
       [ 4.79398288e-01, -2.31186698e-01, -8.46599074e-01],
       [ 5.98220139e-01, -3.79824063e-02, -8.00431135e-01],
       [ 6.05561746e-01,  2.89722826e-01

In [19]:
print(len(features[100][1]))

170


In [20]:
print(features[0][0])


ARTI_ali_riza_ersin_(+).ply


In [21]:
meshPathes

['ARTI_ali_riza_ersin_(+).ply',
 '2024-09-16_14-55-22_回复__visit_to_Germany.ply',
 'ARTI_Sebnem_Ela_Cengizhan_(+).ply',
 'AKTIF__azra_duru_yildiz.ply',
 '2024-07-05_18-59-35_Fwd__Fw__Patient_Data_ziad_Kareem_3_case_5724.ply',
 '2024-09-20_16-42-31_Fwd__Fw__Patient_Data_Maria_charity_1924.ply',
 'ilkim cemal kara 140921KALECAN.ply',
 'ARTI_yigit_ali_boz_2_(+).ply',
 'KALECAN_ecrin_ada_sener-_korse.ply',
 'ARTI_ayse_beren_nebil.ply',
 '2023-10-06_10-11-52_Fw__3D_Model_Jana_sherif_21023__her_2nd_brace___no_new_xray_yet_.ply',
 'FIZIMED_ZEYNEP_BAHSI_(_AKTIF_).ply',
 '2024-01-18_18-28-46_Patient_Data.ply',
 '2024-09-13_10-55-45_Patient_Data_Estela_Soares.ply',
 'FIZIMED_AHMET_KAYA_(_AKTIF_).ply',
 'KALECAN_defne_bekar.ply',
 'KALECAN_asya_guven.ply',
 'KALECAN_deniz_sukruogullari-korse.ply',
 'ozgu ergin.ply',
 'ARTI_Eylul_Acar.ply',
 'LIV_HOSPITAL_Elif_Irem_Sercin.ply',
 'AKTIF_deniz_kiran.ply',
 'Tugba Naz Korkmaz 3.ply',
 'ARTI_sila_dogan.ply',
 'FIZIMED_FURKAN_OZDEN_(_AKTIF_).ply',
 '202